In [21]:
%matplotlib inline
# Importing standard Qiskit libraries and configuring account
from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *

# Loading your IBM Q account(s)
provider = IBMQ.load_account()

from qiskit import QuantumCircuit, execute, Aer
from qiskit.circuit import QuantumRegister, Qubit, Gate, ClassicalRegister
from qiskit.quantum_info import Statevector

# Transpiling stuff
from qiskit.compiler import transpile

#import utils
from collections import OrderedDict

import math
import numpy as np

# Import general libraries (needed for functions)
import time

# Import Qiskit classes
import qiskit
from qiskit.providers.aer import noise
from qiskit.tools.visualization import plot_histogram

from qiskit.test.mock import FakeYorktown

# Import measurement calibration functions
from qiskit.ignis.mitigation.measurement import (complete_meas_cal, tensored_meas_cal,
                                                 CompleteMeasFitter, TensoredMeasFitter)

/opt/conda/lib/python3.7/site-packages/qiskit/providers/ibmq/ibmqfactory.py:192: UserWarning: Timestamps in IBMQ backend properties, jobs, and job results are all now in local time instead of UTC.
  warnings.warn('Timestamps in IBMQ backend properties, jobs, and job results '
ibmqfactory.load_account:WARNING:2020-10-31 13:43:22,464: Credentials are already in use. The existing account in the session will be replaced.


In [1]:
!pip install mitiq
import mitiq

     |████████████████████████████████| 72 kB 4.0 kB/s  eta 0:00:01
  Using cached numpy-1.18.5-cp37-cp37m-manylinux1_x86_64.whl (20.1 MB)
     |████████████████████████████████| 1.6 MB 11.2 MB/s eta 0:00:01
  Using cached protobuf-3.12.4-cp37-cp37m-manylinux1_x86_64.whl (1.3 MB)
  Using cached google_api_core-1.23.0-py2.py3-none-any.whl (91 kB)
  Using cached freezegun-0.3.15-py2.py3-none-any.whl (14 kB)
  Using cached googleapis_common_protos-1.52.0-py2.py3-none-any.whl (100 kB)
  Using cached google_auth-1.23.0-py2.py3-none-any.whl (114 kB)
  Using cached grpcio-1.33.2-cp37-cp37m-manylinux2014_x86_64.whl (3.8 MB)
  Using cached cachetools-4.1.1-py3-none-any.whl (10 kB)
  Using cached pyasn1_modules-0.2.8-py2.py3-none-any.whl (155 kB)
  Using cached rsa-4.6-py3-none-any.whl (47 kB)
  Using cached pyasn1-0.4.8-py2.py3-none-any.whl (77 kB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.2
    Uninstalling numpy-1.19.2:
      Successfully uninstalled numpy-1.19

In [42]:
number_W = 2
beta = 1

provider = IBMQ.get_provider(hub='ibm-q-research', group='Pablo-Antonio-Mo', project='main') 
print(provider)
backends = provider.backends()
for backend in backends:
    print(backend.name())
    
backend = provider.backends(name = 'ibmq_valencia')[0]
#backend = provider.backends(name = 'ibmq_casablanca')[0]
properties = backend.properties() 
coupling_map = backend.configuration().coupling_map

#backend = FakeYorktown()


<AccountProvider for IBMQ(hub='ibm-q-research', group='Pablo-Antonio-Mo', project='main')>
ibmq_qasm_simulator
ibmqx2
ibmq_16_melbourne
ibmq_vigo
ibmq_ourense
ibmq_valencia
ibmq_armonk
ibmq_athens
ibmq_rome
ibmq_santiago
ibmq_bogota
ibmq_casablanca


In [43]:
backend

<IBMQBackend('ibmq_valencia') from IBMQ(hub='ibm-q-research', group='Pablo-Antonio-Mo', project='main')>

In [24]:
import json
with open('delta_energies_glycylglycine_1_hardware.json') as json_file:
    data = json.load(json_file)

deltas_dictionary = data['deltas']
deltas_dictionary = OrderedDict(sorted(deltas_dictionary.items()))
print(len(deltas_dictionary))
deltas = {}
for (key,value) in deltas_dictionary.items():
    deltas[key[:3]] = value
    #print(key,value)
print('corrected ------------------')
for (key,value) in deltas.items():
    print(key, value)

16
corrected ------------------
000 0.27042123529724904
001 26.032737333542684
010 0.24282032166763656
011 -26.032737333542684
100 -0.27042123529724904
101 26.00513641991307
110 -0.24282032166763656
111 -26.00513641991307


In [25]:
# Bitmap calculation
def calculate_angles(deltas_dictionary, beta = 1):
    
    exact_angles = {}

    for key in deltas_dictionary.keys():

        if deltas_dictionary[key] >= 0:
            probability = math.exp(-beta * deltas_dictionary[key])
        else: 
            probability = 1
        # Instead of encoding the angle corresponding to the probability, we will encode the angle theta such that sin^2(pi/2 - theta) = probability.
        # That way 1 -> 000, but if probability is 0 there is some small probability of acceptance

        # Instead of probability save angles so rotations are easier to perform afterwards sqrt(p) = sin(pi/2-theta/2).
        # The theta/2 is because if you input theta, qiskits rotates theta/2. Also normalised (divided between pi the result)
        exact_angles[key] = math.pi - 2 * math.asin(math.sqrt(probability))

    # Order angles by key
    exact_angles = OrderedDict(sorted(exact_angles.items()))

    return exact_angles

Let us have some error correction in place

In [26]:
'''
# Generate the calibration circuits
coin = QuantumRegister(1, name = 'coin')
move_id  = QuantumRegister(1, name = 'move_id')
angle_psi = QuantumRegister(1, name = 'angle_psi')
angle_phi = QuantumRegister(1, name = 'angle_phi')
# By order those qubits will be qr[0], qr[1], qr[2] abnd qr[3]


#qubit_list = [angle_phi[0], angle_psi[0], move_id[0], coin[0]]
qr = qiskit.QuantumRegister(4)
qubit_list = [0,1,2,3]
#
meas_calibs, state_labels = complete_meas_cal(qubit_list=qubit_list, qr = qr, circlabel='mcal')

layout = {3: qr[3], 6: qr[2], 4: qr[1], 5: qr[0]}
# layout = {2: angle_psi[0], 3: angle_phi[0], 1: coin[0], 0: move_id[0]} # This is another possible configuration
job = qiskit.execute(meas_calibs, backend=backend, shots=1024, initial_layout=layout)
cal_results = job.result()

# Calculate the calibration matrix with the noise model
meas_fitter = CompleteMeasFitter(cal_results, state_labels, qubit_list=qubit_list, circlabel='mcal')

# Plot the calibration matrix
meas_fitter.plot_calibration()
'''

"\n# Generate the calibration circuits\ncoin = QuantumRegister(1, name = 'coin')\nmove_id  = QuantumRegister(1, name = 'move_id')\nangle_psi = QuantumRegister(1, name = 'angle_psi')\nangle_phi = QuantumRegister(1, name = 'angle_phi')\n# By order those qubits will be qr[0], qr[1], qr[2] abnd qr[3]\n\n\n#qubit_list = [angle_phi[0], angle_psi[0], move_id[0], coin[0]]\nqr = qiskit.QuantumRegister(4)\nqubit_list = [0,1,2,3]\n#\nmeas_calibs, state_labels = complete_meas_cal(qubit_list=qubit_list, qr = qr, circlabel='mcal')\n\nlayout = {3: qr[3], 6: qr[2], 4: qr[1], 5: qr[0]}\n# layout = {2: angle_psi[0], 3: angle_phi[0], 1: coin[0], 0: move_id[0]} # This is another possible configuration\njob = qiskit.execute(meas_calibs, backend=backend, shots=1024, initial_layout=layout)\ncal_results = job.result()\n\n# Calculate the calibration matrix with the noise model\nmeas_fitter = CompleteMeasFitter(cal_results, state_labels, qubit_list=qubit_list, circlabel='mcal')\n\n# Plot the calibration matrix\

Let us first construct the substitute of the oracle

In [27]:
angles = calculate_angles(deltas_dictionary = deltas, beta = .1)
print(angles)
def hardware_GG_1_coin_flip(circuit,coin,move_id,angle_psi,angle_phi,angles,inv,iteration,beta):
    '''Warning! This only works for GG 1 in hardware mode. Do not use elsewhere!'''
    # First we have to identify the non-zero angles. For the rest we accept with probability 1
    circuit.x(coin)
    '''
    Since the angles from 001 and 101 ~= 2.59; and those from 010 and 000 ~= 0.32 (when beta = .1, 
    but they'll always be similar nevertheless), we will perform those rotations together
    '''
    non_zero_angles = {}
    non_zero_angles['0x0'] = (angles['000']+angles['010'])/2
    non_zero_angles['x01'] = (angles['001']+angles['101'])/2
    
    # Let us first perform the first
    circuit.x(angle_phi)
    circuit.x(move_id)
    circuit.mcrx(theta = -inv*non_zero_angles['0x0'],
                 q_controls = [move_id[0],angle_phi[0]], q_target = coin[0], use_basis_gates=True)
    circuit.x(angle_phi)
    circuit.x(move_id)
    
    # Let us perform the second
    circuit.x(angle_psi)
    circuit.mcrx(theta = -inv*non_zero_angles['x01'],
                 q_controls = [move_id[0],angle_psi[0]], q_target = coin[0], use_basis_gates=True)
    circuit.x(angle_psi)

OrderedDict([('000', 0.328149137182979), ('001', 2.5904722228055745), ('010', 0.3110236124064589), ('011', 0.0), ('100', 0.0), ('101', 2.58969121534896), ('110', 0.0), ('111', 0.0)])


The quantum Walk is composed of several parts, $$W = R V^\dagger B^\dagger F B V,$$
where $V$ prepares possible moves, $B$ is the Boltzmann coin, $F$ the coin flip, and $R$ a reflection of all registers except the state one over state $\ket{0}$. The first thing we do is prepare the oracle


In [28]:
def W_step(qc,coin,move_id,angle_psi,angle_phi,angles,iteration,beta): 
    # Perform the preparation of possible moves----
    qc.h(move_id)

    # Prepare the Boltzmann coin ------------------
    hardware_GG_1_coin_flip(qc,coin,move_id,angle_psi,angle_phi,angles,inv = 1,iteration=iteration,beta = beta)
    
    # Perform move ---------------------------------
    # For the second angle
    qc.ccx(coin,move_id,angle_psi)

    # For the first angle
    qc.x(move_id)
    qc.ccx(coin,move_id,angle_phi)
    qc.x(move_id)

    if iteration == 0:
        # Unprepare the Boltzmann coin--------------------
        hardware_GG_1_coin_flip(qc,coin,move_id,angle_psi,angle_phi,angles,inv = -1,iteration=0,beta = beta)

        # Perform the preparation of possible moves ----
        qc.h(move_id)

        #Reflection -------------------------------------
        qc.x(move_id)
        qc.x(coin)

        # Perform a multicontrolled Z
        qc.cz(move_id,coin)

        qc.x(move_id)
        qc.x(coin)

Running the model on a backend

In [29]:
def generate_circ(betas = [.1,1]):
    
    move_id  = QuantumRegister(1)
    angle_phi = QuantumRegister(1)
    angle_psi = QuantumRegister(1)
    coin = QuantumRegister(1)
    c_reg = ClassicalRegister(2)
    qc = QuantumCircuit(coin,move_id,angle_psi,angle_phi,c_reg)

    #Circuit ----------
    qc.h(angle_phi)
    qc.h(angle_psi)
    for (i,beta) in zip(range(number_W),betas):
        angles = calculate_angles(deltas_dictionary = deltas, betas = betas)
        W_step(qc,coin,move_id,angle_psi,angle_phi,angles,i,beta = beta)

    # Measure
    qc.measure(angle_phi[0], c_reg[1])
    qc.measure(angle_psi[0], c_reg[0])

    # Transpiling -------

    #layout = {5: angle_phi[0], 6: angle_psi[0], 4: move_id[0], 5: coin[0]}
    layout = {2: angle_psi[0], 3: angle_phi[0], 1: coin[0], 0: move_id[0]} 
    qc = transpile(qc, backend = backend, optimization_level=3, 
                   initial_layout=layout, basis_gates = ['u1', 'u2', 'u3', 'cx'], routing_method = 'lookahead')
    print('After optimization--------')
    print('gates = ', qc.count_ops())
    print('depth = ', qc.depth())
    return qc

We can first execute the same circuit without error

In [30]:
def exe_noiseless(betas = betas):

    move_id  = QuantumRegister(1, name = 'move_id')
    angle_phi = QuantumRegister(1, name = 'angle_phi')
    angle_psi = QuantumRegister(1, name = 'angle_psi')
    coin = QuantumRegister(1, name = 'coin')
    c_reg = ClassicalRegister(4)
    aerqc = QuantumCircuit(coin,move_id,angle_psi,angle_phi,c_reg)

    #Circuit ----------
    aerqc.h(angle_phi)
    aerqc.h(angle_psi)
    for (i,beta) in zip(range(number_W),betas):
        angles = calculate_angles(deltas_dictionary = deltas, betas = betas)
        W_step(aerqc,coin,move_id,angle_psi,angle_phi,angles,i,beta)

    # Measure
    aerbackend = Aer.get_backend('statevector_simulator')
    backend_options = {"method" : "statevector"}
    experiment = execute(aerqc, aerbackend, backend_options=backend_options)
    state_vector = Statevector(experiment.result().get_statevector(aerqc))

    probabilities = state_vector.probabilities([3,2])
    noiseless_counts = {}
    noiseless_counts['00'] = float(probabilities[0])
    noiseless_counts['01'] = float(probabilities[1])
    noiseless_counts['10'] = float(probabilities[2])
    noiseless_counts['11'] = float(probabilities[3])

    return noiseless_counts
    #plot_histogram(noiseless_counts)
#noiseless_counts = exe_noiseless(beta = 1e-15)
#noiseless_counts

We will do a small experiment to see if we can measure anything different from noise. The initialization is

In [44]:
n_iterations = 10
raw_counts = {'beta=1e-10': [], 'beta=0.1': []}
noiseless_counts = {'beta=1e-10': [], 'beta=0.1': []}

We can calculate what would happen with the exact same circuit (except for parametrised angles) with very small beta

In [45]:
betas = [1e-10,1e-10]
qc = generate_circ(betas = betas)

for i in range(n_iterations):
    print('iteration =',i)
    counts= execute(qc, backend, shots=8192).result().get_counts()
    raw_counts['beta=0'].append(counts['00'])
    
noiseless_counts['beta=0'] = exe_noiseless(beta = beta)['00']

print(np.average(raw_counts['beta=0']))
print(np.std(raw_counts['beta=0']))
#plot = plot_histogram([raw_counts,noiseless_counts], legend=['raw','noiseless'], color = ['red','green'])

After optimization--------
gates =  OrderedDict([('cx', 105), ('u2', 51), ('u3', 44), ('u1', 18), ('measure', 2), ('barrier', 1)])
depth =  173
iteration = 0
iteration = 1
iteration = 2
iteration = 3
iteration = 4
iteration = 5
iteration = 6
iteration = 7
iteration = 8
iteration = 9
1919.5
30.735159020249107


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:21: DeprecationWarning: Using `backend_options` kwarg has been deprecated as of qiskit-aer 0.7.0 and will be removed no earlier than 3 months from that release date. Runtime backend options should now be added directly using kwargs for each option.


Then, we choose the right beta to correct

In [46]:
betas = [0.1,1]
qc = generate_circ(betas = betas)

for i in range(n_iterations):
    print('iteration =',i)
    counts= execute(qc, backend, shots=8192).result().get_counts()
    raw_counts['beta=1'].append(counts['00'])
    
noiseless_counts['beta=1'] = exe_noiseless(betas = betas)['00']

print(np.average(raw_counts['beta=1']))
print(np.std(raw_counts['beta=1']))

#plot = plot_histogram([raw_counts,noiseless_counts], legend=['raw','noiseless'], color = ['red','green'])

After optimization--------
gates =  OrderedDict([('cx', 105), ('u2', 50), ('u3', 45), ('u1', 18), ('measure', 2), ('barrier', 1)])
depth =  173
iteration = 0
iteration = 1
iteration = 2
iteration = 3
iteration = 4
iteration = 5
iteration = 6
iteration = 7
iteration = 8
iteration = 9
2008.7
51.60242242375836


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:21: DeprecationWarning: Using `backend_options` kwarg has been deprecated as of qiskit-aer 0.7.0 and will be removed no earlier than 3 months from that release date. Runtime backend options should now be added directly using kwargs for each option.


In [62]:
import scipy
def generate_bernouilli(n_0,n):
    array = np.random.binomial(1,n_0/n,n)
    s = np.sum(array)
    while s != n_0:
        i = np.random.randint(n)
        if s<n_0 and array[i] == 0:
            array[i] = 1
        elif s>n_0 and array[i] == 1:
            array[i] = 0
        s = np.sum(array)
    return array    
beta0_bernouilli = generate_bernouilli(np.sum(raw_counts['beta=0']), 81920)
beta1_bernouilli = generate_bernouilli(np.sum(raw_counts['beta=1']), 81920)
scipy.stats.ttest_ind(beta0_bernouilli, beta01_bernouilli, equal_var=False)
#scipy.stats.ttest_ind(raw_counts['beta=1e-10'], raw_counts['beta=0.1'], equal_var=False)

Ttest_indResult(statistic=-5.162081762713354, pvalue=2.4450295754288155e-07)

In [55]:
def executor(circuit: qiskit.QuantumCircuit, shots: int = 8192) -> float:
    """Returns the expectation value to be mitigated.

    Args:
        circuit: Circuit to run.
        shots: Number of times to execute the circuit to compute the expectation value.
    """

    # (1) Run the circuit
    raw_counts = 0
    for i in range(n_iterations):
        print('iteration =',i)
        counts= execute(qc, backend, shots=shots, optimization_level=0).result().get_counts()
        raw_counts += counts['00']
        
    # (2) Convert from raw measurement counts to the expectation value    
    expectation_value = raw_counts/(shots*n_iterations)

    return expectation_value


betas = [0.1,1]
qc = generate_circ(betas = betas)
linear_factory = mitiq.zne.inference.LinearFactory(scale_factors=[1.0, 1.05, 1.1, 1.15, 1.2, 1.25])
mitigated = mitiq.execute_with_zne(qc, executor, fac=linear_factory)

{'beta=1e-10': [1902, 1944, 1958, 1928, 1867, 1886, 1915, 1891, 1961, 1943],
 'beta=0.1': [2050, 1981, 1985, 2008, 1986, 2111, 2066, 1989, 1915, 1996]}

beta = 0, theoretical_expected_value = 0.25

1. Ourense (1450.6, 68.7447452537283)

2. Valencia (1919.5, 30.735159020249107)

beta = 0.1, theoretical_expected_value = 0.48

1. Ourense (1650.7, 87.46204891265697)

2. Valencia (2008.7, 51.60242242375836)

In [ ]:
def success_rate(cnot_error, u_error, cnot_number, u3_number, u2_number, T1s, T2s, circ_depth):
    # Calculate gate success rate
    gate_success = (1-cnot_error)**cnot_number*(1-u_error)**(u3_number + u2_number)
    
    # Calculate execution time
    T = 0.8*cnot_numer + 0.3*u3_number +0.15*(circ_depth - cnot_number - u3_number)
    
    s = 1
    
    # Either:
    #for (T1,T2) in zip(T1s,T2s):
    #   s *= math.exp(-T/T1)*math.exp(-T/T2)
        
    # Or, since T2 < T1
    for T2 in T2s:
        s *= math.exp(-T/T2)
        
    return s*gate_success